# Лабораторная работа № 5
### Минимальная оценка - 3 балла
### Максимальная оценка - 5 балла
Цель работы: реализация метрических алгоритмов машинного обучения.

Для успешной сдачи лабораторной работы Вам необходимо предоставить заполненый исходный ноутбук, продемострировать процесс выполнения работы и ответить на дополнительные вопросы преподавателя.

### Описание датасета:

Данный датасет посвящён определению типа опухоли (доброкачественная или злокачественная).
- id - идентификатор пользователя 
- diagnosis - диагноз (M = malignant (злокачественная опухоль), B = benign (доброкачественная)) - является целевой переменной 
- столбцы 3-33 - параметры опухоли. 

## Задание 1 (3 балла)

### Задание 1.1
Загрузите датасет $breast\_cancer\_wisconsin\_data.csv$, выполните его предобработку, разделите на обучающую и тестовую выборки (в соотношении 7:3) и выделите метки классов.

*Подсказка: некоторые неинформативные поля можно удалить.*

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv("breast_cancer_wisconsin_data.csv")
df.shape

(569, 33)

In [5]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [6]:
X, y = df.drop(["Unnamed: 32", "diagnosis", "id"], axis=1), df["diagnosis"].map({"M": 0,"B": 1}) 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((398, 30), (171, 30), (398,), (171,))

### Задание 1.2

Пользуясь материалами лекции, реализуйте свой алгоритм **kNN**, который содержит методы:
- **fit()**, который принимает в качестве входного параметра обучающую выборку и хранит её в переменной $self.data$, а также число ближайших оседей ($k$), которое должно храниться в переменной $self.k$.
- **predict()**, который принимает в качестве входного параметра тестовую выборку и возвращает метки классов для неё ($predict\_y$).

*Метод $predict()$ должен для каждого объекта тестовой выборки посчитать расстояние до всех объектов обучающей выборки, выбрать $k$ ближайших (по расстоянию) объектов и, по меткам их классов, назначить метку класса для текущего объекта тестовой выборки.*

В качестве метрики расстояния между объектами используйте [евклидово расстояние](http://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html).

In [8]:
class knn(object):
    def __init__(self, k=3):
        self.k = k
        self._X, self._y = None, None
    
    def fit(self, X_train, y_train):
        self._X = np.array(X_train)
        self._y = np.array(y_train)
        
    def predict(self, X):
        y_pred = np.zeros(X.shape[0])
        X = np.array(X)
        
        if X.ndim == 1:
            X = X.reshape(1, -1)
            
        for i, obj in enumerate(X):
            d = np.linalg.norm(obj - self._X, axis=1)
            neighbors = np.argsort(d)[:self.k]
            y_pred[i] = np.bincount(self._y[neighbors]).argmax()
            
        return y_pred
    
    def get_params(self, deep=True):
        return {"k": self.k}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
            
        return self

Продемонстрируйте работу получившегося классификатора на Ваших выборках для разных значений параметра $k$ (1,5,10).

In [9]:
ks = [1, 5, 10]

models = list(map(knn, ks))

In [10]:
for m in models:
    m.fit(X_train, y_train)

In [11]:
y_preds = [m.predict(X_test) for m in models]

In [12]:
n_show = 10
y_preds[0][:n_show], y_preds[1][:n_show], y_preds[2][:n_show]

(array([1., 0., 0., 1., 1., 0., 0., 0., 1., 1.]),
 array([1., 0., 0., 1., 1., 0., 0., 0., 1., 1.]),
 array([1., 0., 0., 1., 1., 0., 0., 0., 1., 1.]))

### Задание 1.3

Реализуйте любые 2 метрики оценки качества классификации (из рассмотренных на лекции) и, используя их, сравните Ваши предсказания ($predict\_y$) с тестовыми метками классов.
Результат выведете в отдельной ячейке в формате 'k=n, metric_name : score'. Например: $$k=1, accuracy: 0.756$$

In [13]:
def recall_score(y_true, y_pred, *, label=1):
    P  = sum([t == label for t in y_true])
    TP = sum([t == p for t, p in zip(y_true, y_pred) if t == label])
    
    return TP / P

In [14]:
def precision_score(y_true, y_pred, *, label=1):
    TP = sum([t == p for t, p in zip(y_true, y_pred) if t == label])
    FP = sum([t != p for t, p in zip(y_true, y_pred) if t != label])
    
    return TP / (TP + FP)

In [15]:
recalls    = [recall_score(y_test.values, y_pred) for y_pred in y_preds]
precisions = [precision_score(y_test.values, y_pred) for y_pred in y_preds]

In [16]:
for (k, recall, precision) in zip(ks, recalls, precisions):
    print(f'{k = }:\t{recall = }, {precision = }')

k = 1:	recall = 0.9629629629629629, precision = 0.9369369369369369
k = 5:	recall = 0.9907407407407407, precision = 0.9469026548672567
k = 10:	recall = 0.9907407407407407, precision = 0.981651376146789


## Задание 2 (1 балл)

### Задание 2.1

Реализуйте метод Парзеновского окна фиксированной ширины и продемонстрируйте его работу на ваших выборках.

In [17]:
# ваш код

### Задание 2.2

Используя метрики из **задания 1.3**, сравните Ваши предсказания ($predict\_y$) с тестовыми метками классов.
Результат выведете в отдельной ячейке в формате 'h=n, metric_name : score'. Например: $$h=1, accuracy: 0.756$$

In [18]:
# ваш код

## Задание 3 (1 балл)

Оптимизируйте параметр $k$ алгоритма **kNN** по критерию скользящего контроля (leave-one-out). При поиске оптимального значения брать только нечётные значения $k$.

In [19]:
from sklearn.model_selection import (LeaveOneOut,
                                     GridSearchCV)

In [20]:
param_grid = {'k': [2 * i + 1 for i in range(10)]}

In [21]:
gs = GridSearchCV(estimator=knn(), 
                  param_grid=param_grid,
                  scoring={'recall': recall_score, 
                           'precision': precision_score}, 
                  cv=LeaveOneOut(), 
                  refit='recall')

In [22]:
gs.fit(X_train.values, y_train.values)

GridSearchCV(cv=LeaveOneOut(), estimator=<__main__.knn object at 0x10c11f280>,
             param_grid={'k': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]},
             refit='recall',
             scoring={'precision': <function precision_score at 0x10c1308b0>,
                      'recall': <function recall_score at 0x10c130b80>})

In [23]:
y_pred = gs.predict(X_test)

In [24]:
recall = recall_score(y_test.values, y_pred)
precision = precision_score(y_test.values, y_pred)

In [25]:
print(f'{recall    = }\n{precision = }')

recall    = 0.9629629629629629
precision = 0.9369369369369369
